In [1]:
import pandas as pd
import zipfile
from pathlib import Path
import json
pd.set_option('display.max_rows', 50)
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
DF_INDEX_PATH = Path("index_df.zip")
MITRE_LABELS_JSON_PATH = Path("MITRE_labels.json")

In [3]:
index_df = pd.read_csv(DF_INDEX_PATH)
print(index_df.shape)
index_df[(index_df.group_name != "[]") & (index_df.software_name != "[]")& (index_df.technique_name != "[]")]

(657724, 8)


,Unnamed: 0,git_repo,year,filename,group_name,software_name,technique_name,tactic
861,861,APT_CyberCriminal_Campagin_Collections,2018,ESET_Turla_Mosquito.pdf.txt,['turla'],['uroburos'],['rootkit'],[]
20047,20047,APT_CyberCriminal_Campagin_Collections,2018,ukatemicrysys_territorialdispute.pdf.txt,['turla'],['uroburos'],['rootkit'],[]
439827,439827,APT_CyberCriminal_Campagin_Collections,2015,Satellite-turla-apt-command-and-control-in-the...,['turla'],['uroburos'],['rootkit'],[]
626948,626948,APT_CyberCriminal_Campagin_Collections,2019,ESET_Operation_Ghost_Dukes.pdf.txt,['apt29'],['poshspy'],['powershell'],[]
627108,627108,APT_CyberCriminal_Campagin_Collections,2019,ESET_Operation_Ghost_Dukes.pdf.txt,['the dukes'],['psexec'],['service execution'],[]
627154,627154,APT_CyberCriminal_Campagin_Collections,2019,ESET_Operation_Ghost_Dukes.pdf.txt,['the dukes'],['psexec'],['windows admin shares'],[]


In [4]:
labels = ["group_name", "software_name", "technique_name", "tactic"]
print(f"Total lines {len(index_df)}")
for label in labels:
    print(f"lines with {label}: {len(index_df[index_df[label] != str([])])}")

Total lines 657724
lines with group_name: 11132
lines with software_name: 16492
lines with technique_name: 8132
lines with tactic: 8474


In [5]:
year_df = pd.DataFrame()

year_df["year"]= index_df["year"].value_counts(sort=True).sort_index().index
year_df["count"] = index_df["year"].value_counts(sort=True).sort_index().values
year_df["%"] = index_df["year"].value_counts(sort=True, normalize=True).sort_index().values


year_df

,year,count,%
0,2006,251,0.00
1,2008,4725,0.01
2,2009,5916,0.01
3,2010,10706,0.02
4,2011,7948,0.01
5,2012,17644,0.03
6,2013,58956,0.09
7,2014,72915,0.11
8,2015,74365,0.11
9,2016,94384,0.14


In [6]:
# slice into interesting lines only -> than create a column for each name the dataset was labeled by
verbose_index_df = index_df.copy()

verbose_index_df = verbose_index_df[
    (verbose_index_df.group_name != "[]")    |
    (verbose_index_df.software_name != "[]") |
    (index_df.technique_name != "[]")        |
    (index_df.tactic != "[]") 
]
print(f"{verbose_index_df.shape=}")

def load_mitre_labels_lower():
    with Path(MITRE_LABELS_JSON_PATH).open("r", encoding="utf8") as f:
        mitre_labels = json.load(f)
        for entity_type in mitre_labels:
            mitre_labels[entity_type] = [name.lower() for name in mitre_labels[entity_type]]
    return mitre_labels

def check_if_label_exists(label, line):
    if label in line:
        return True
    return False

mitre_labels = load_mitre_labels_lower()
values_list = []
for entity_type in mitre_labels:
    for name in mitre_labels[entity_type]:
        values_list.append(name)
        verbose_index_df[name] = index_df[entity_type].str.contains(name)

verbose_index_df

verbose_index_df.shape=(42393, 8)


,Unnamed: 0,git_repo,year,filename,group_name,software_name,technique_name,tactic,energetic bear,chinastrats,...,defense evasion,collection,credential access,execution,lateral movement,discovery,command and control,persistence,exfiltration,privilege escalation
51,51,APT_CyberCriminal_Campagin_Collections,2018,MuddyWater_Middle_East_and_Central_Asia.pdf.txt,[],[],['powershell'],[],False,False,...,False,False,False,False,False,False,False,False,False,False
128,128,APT_CyberCriminal_Campagin_Collections,2018,MuddyWater_Middle_East_and_Central_Asia.pdf.txt,[],[],[],['privilege escalation'],False,False,...,False,False,False,False,False,False,False,False,False,True
131,131,APT_CyberCriminal_Campagin_Collections,2018,MuddyWater_Middle_East_and_Central_Asia.pdf.txt,[],[],['powershell'],[],False,False,...,False,False,False,False,False,False,False,False,False,False
134,134,APT_CyberCriminal_Campagin_Collections,2018,MuddyWater_Middle_East_and_Central_Asia.pdf.txt,[],[],['powershell'],[],False,False,...,False,False,False,False,False,False,False,False,False,False
135,135,APT_CyberCriminal_Campagin_Collections,2018,MuddyWater_Middle_East_and_Central_Asia.pdf.txt,[],[],[],['persistence'],False,False,...,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
657633,657633,APT_CyberCriminal_Campagin_Collections,2008,chinas-electronic.pdf.txt,[],['page'],[],[],False,False,...,False,False,False,False,False,False,False,False,False,False
657639,657639,APT_CyberCriminal_Campagin_Collections,2008,chinas-electronic.pdf.txt,[],[],['source'],[],False,False,...,False,False,False,False,False,False,False,False,False,False
657665,657665,APT_CyberCriminal_Campagin_Collections,2008,army-bans-usb-d.pdf.txt,[],[],[],['discovery'],False,False,...,False,False,False,False,False,True,False,False,False,False
657717,657717,APT_CyberCriminal_Campagin_Collections,2008,army-bans-usb-d.pdf.txt,[],[],[],['collection'],False,False,...,False,True,False,False,False,False,False,False,False,False


In [7]:
def count_entity_values(verbose_index_df, entity_type):
    value_counter = dict()
    for name in mitre_labels[entity_type]:
        value_counter[name] = [verbose_index_df[name].sum()]

    with pd.option_context('display.max_rows', 500, 'display.max_columns', 10):
        df = pd.DataFrame(value_counter).transpose()
        print(df.shape)
        df.index.name = entity_type
        df.columns = ["count"]
        display(df)


In [8]:
count_entity_values(verbose_index_df, entity_type='group_name')

(112, 1)


,count
group_name,
energetic bear,57
chinastrats,4
gamaredon group,136
hidden cobra,74
monsoon,83
fancy bear,112
stone panda,39
buckeye,71
threat group-1314,0


In [9]:
count_entity_values(verbose_index_df, entity_type='software_name')

(183, 1)


,count
software_name,
hammerduke,32
nbtstat.exe,0
ned worm,19
xtunnel,101
sys10,19
hcdloader,1
enfal,47
elmer,14
miniduke,367


In [10]:
count_entity_values(verbose_index_df, entity_type='technique_name')

(168, 1)


,count
technique_name,
code signing,157
access token manipulation,8
re-opened applications,0
shortcut modification,7
service registry permissions weakness,0
pass the ticket,5
input prompt,2
execution through module load,8
network sniffing,8


In [11]:
count_entity_values(verbose_index_df, entity_type='tactic')

(10, 1)


,count
tactic,
defense evasion,63
collection,617
credential access,47
execution,2643
lateral movement,429
discovery,943
command and control,1579
persistence,1488
exfiltration,657
